## Soccertrack - Play Evaluation
 
This is a guide for evaluating split play data.

Currently, play evaluation using xG and xT indicators is available; OBSO is in the process of implementation and will be added soon.

Please note that the evaluation indicators use pre-computed parameters. Learning from scratch, for example, is outside the scope of this guide.

The xG and xT parameters were referenced below👓

- xG : [An examination of expected goals and shot efficiency in soccer](https://pdfs.semanticscholar.org/ed25/5fbf1382588c8ab996af3690ef209323ac63.pdf?_ga=2.54952171.1368408628.1675579413-2014574046.1675036569)
- xT : [Introducing Expected Threat (xT)](https://karun.in/blog/expected-threat.html)

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import pickle
import os
import numpy as np
import pandas as pd

from pathlib import Path
from soccertrack.dataframe import CoordinatesDataFrame
from soccertrack.rate import rate_xG, rate_xT
from soccertrack.logger import show_df, tqdm
from soccertrack.utils import get_git_root

## Part1. Calculation of xG & xT
A script that downloads all the data from labelbox is available in `soccertrack/rate/rate.py`. 

In [27]:
#setup initial variables
root = get_git_root()
video_dir = Path(root / "soccertrack" / "datasets" / 'SoccerData' / 'TASC' / 'out_videos')
data_path = Path(root / "soccertrack" / "datasets" / 'SoccerData' / 'train' / 'data.pkl')
metrics_path = Path(root/ "soccertrack" / "datasets" / 'SoccerData' / 'metrics_list.csv')
metrics_df = pd.read_csv(metrics_path)

#Window to divide pitch by grid
window_x = 16
window_y = 12

#load tracking data
with open(data_path, 'rb') as f:
    data = pickle.load(f)

#transform to CoordinatesDataFrame and store in list
#Todo 今後のことを考えると、プレーデータをlistにしておくのは良くないかもしれないので、ジェネレーターにするなどの対応を考える
codf_list = []
for item in tqdm(data.items(), desc="Transform to CoordinatesDataFrame.."):
    idx, seq = item
    codf = CoordinatesDataFrame.from_numpy(seq)
    codf_list.append({"idx": idx, "data": codf})

Transform to CoordinatesDataFrame..: 100%|██████████| 6000/6000 [00:07<00:00, 850.88it/s]


### Calculation of xG

In [33]:
from soccertrack.utils import make_video
sample_codf = codf_list[0]

xg_mtx = metrics_df['xG_area'].to_numpy().reshape(window_x, window_y)
rate_xG(sample_codf['data'], xg_mtx, agg_func="w_mean")

0.02798701298701299

### Calculation of xT

In [10]:
sample_codf = codf_list[0]
xt_mtx = metrics_df['xT_area'].to_numpy().reshape(window_x, window_y)

rate_xT(sample_codf['data'], xt_mtx, agg_func="w_mean")

0.02927922077922078

### Sorting play quality by xG

In [12]:
#sort play quality by xG
codf_list_sort = sorted(
                codf_list, 
                key=lambda codf: rate_xG(codf['data'], xg_mtx, agg_func="w_mean"), 
                reverse=True
                )

sorted_codf_idx = [codf_sort['idx'] for codf_sort in codf_list_sort]

/home/guest/dev_repo/SoccerTrack/soccertrack/rate/agg_func.py:7: RuntimeWarning: invalid value encountered in scalar divide
  return lambda metrics_mtx, ma_count_arr: np.sum(metrics_mtx * ma_count_arr) / np.sum(ma_count_arr)
